In [53]:
#--Dependencies--#

#-- Data Cleaning Libraries:
import pandas as pd
import numpy as np

#-- Data Visualization Libraries:
from matplotlib import pyplot as plt
import seaborn as sns #--just in case we need it, probably won't


#-- Web Scraping Libraries:
import os
import time
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser


In [45]:
#Settings for accessing Website

executable_path = {"executable_path": "chromedriver.exe"}

browser = Browser ('chrome', **executable_path, headless=False)



In [19]:
#URL's for both NBA and NCAA

NBA_url = "https://www.basketball-reference.com/"

NCAA_url = "https://www.sports-reference.com/cbb/"


In [20]:
#Create a url_list
url_list = [NBA_url, NCAA_url]

browser.visit(url_list[0])



In [41]:
#Create empty DataFrames to merge player data

In [42]:
#Define function for merging data (to be called in function for scraping: scrape_data) 

In [54]:
#Define our function for data scraping

def scrape_data(our_urls):

    for url in our_urls:
        
        #Visit NBA url
        if url == NBA_url:
            print(f"NBA:{url}")
            browser.visit(url)
            html = browser.html
            soup = bs(html, 'html.parser')
            print(soup.prettify)
            print("------------------------------------------------------------------")
            
        #Visit NCAA url
        else:
            print(f"NCAA:{url}")
            browser.visit(url)
            html = browser.html
            soup = bs(html, 'html.parser')
            print(soup.prettify)
        
scrape_data(url_list)

NBA:https://www.basketball-reference.com/
<bound method Tag.prettify of <html class=" js cookies cors history localstorage sessionstorage canvas no-touchevents unicode pointerevents vibrate matchmedia flexwrap desktop is_live is_modern overthrow-enabled" data-root="/home/bbr/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en" style=""><head id="suppress_all_ads">
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/201907110" rel="dns-prefetch"/>
<!-- yes:cookie regular load the cached css -->
<link crossorigin="" href="https://d2p3bygnnzw9w3.cloudfront.net" rel="preconnect"/>
<link crossorigin="" href="https://d2cwpp38twqe55.cloudfront.net" rel="preconnect"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/201907131/css/bbr/sr-min.css" rel="stylesheet" type="text/css"/>
<l

<bound method Tag.prettify of <html class=" js cookies cors history localstorage sessionstorage canvas no-touchevents unicode pointerevents vibrate matchmedia flexwrap desktop is_live is_modern overthrow-enabled" data-root="/home/cbb/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en" style=""><head id="suppress_all_ads">
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/201907110" rel="dns-prefetch"/>
<!-- yes:cookie regular load the cached css -->
<link crossorigin="" href="https://d2p3bygnnzw9w3.cloudfront.net" rel="preconnect"/>
<link crossorigin="" href="https://d17lgqwvsissft.cloudfront.net" rel="preconnect"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/201907131/css/cbb/sr-min.css" rel="stylesheet" type="text/css"/>
<link as="script" crossorigin="" href="https